# **BIG DATA - L14**

**NHÓM 10**
1. Hồ Thị Thu Ngân - 050608200463
2. Bùi Thị Thanh Thảo - 050608200145 

## **HOMEWORK 1**

# **CÂU 1**

In [ ]:
# Cài đặt Spark 
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Avoids scroll-in-the-scroll in the entire Notebook
from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 400})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

<IPython.core.display.Javascript object>

Mounted at /content/drive


In [ ]:
# Nhập các thư viện cần dùng 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf 

<IPython.core.display.Javascript object>

In [ ]:
from pyspark.sql import SparkSession 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def line2dataset(line):
    src, dst_line= line.split('\t')
    src = int(src.strip())
    dst_list = [int(x.strip()) for x in dst_line.split(',') if x != '']
    return src, dst_list

In [ ]:
def filter_pairs(x):
    if (x[0][0] != x[1][0]) and (not x[0][0] in x[1][1]) and (not x[1][0] in x[0][1]):
        shared = len(list(set(x[0][1]).intersection(set(x[1][1]))))
        return (x[0][0],[x[1][0],shared])

In [ ]:
def map_finaldataset(elem):
    src = elem[0]
    dst_commons = elem[1]
    dst_commons=sorted(dst_commons,key=lambda x:(-x[1],x[0]))[:10]
    recommendations=[pair[0] for pair in dst_commons]
    return (src, recommendations)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
if __name__ == "__main__":
    spark = SparkSession.builder.getOrCreate()
    sc = spark.sparkContext
    dataset = sc.textFile("/content/soc-LiveJournal1Adj.txt")

    dataset = dataset.map(line2dataset)

    check_users = [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]
    cartesian = dataset.cartesian(dataset).filter(lambda x: x[0][0] in check_users)

    dataset = cartesian.map(filter_pairs).filter(lambda x: x != None and x[1][1] > 0)\
        .filter(lambda x: x[0] in check_users) \
        .groupByKey().mapValues(list).map(map_finaldataset)

    id_check_dataset = dataset.filter(lambda x: x[0] in check_users).collect()

    for key, val in id_check_dataset:
        print('id:', key,' recommendations:', val) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

id: 9020  recommendations: [9021, 9016, 9017, 9022, 317, 9023]
id: 924  recommendations: [439, 2409, 6995, 11860, 15416, 43748, 45881]
id: 9992  recommendations: [9987, 9989, 35667, 9991]
id: 9021  recommendations: [9020, 9016, 9017, 9022, 317, 9023]
id: 9993  recommendations: [9991, 13134, 13478, 13877, 34299, 34485, 34642, 37941]
id: 8941  recommendations: [8943, 8944, 8940]
id: 9022  recommendations: [9019, 9020, 9021, 317, 9016, 9017, 9023]
id: 9990  recommendations: [13134, 13478, 13877, 34299, 34485, 34642, 37941]
id: 8942  recommendations: [8939, 8940, 8943, 8944]
id: 9019  recommendations: [9022, 317, 9023]


# **Câu 2**

**Câu a:**

*Nhược điểm:* Confidence có sự bất cập là vì nó đã bỏ qua xác suất của Pr(B). Đôi khi Confidence có kết quả cao do tập mục A của luật kết hợp tương đối cao (ví dụ một tiệm tạp hóa bán quá nhiều A nhưng B lại ít thì Confidence tương đối cao), mà không phải do có mối quan hệ giữa 2 tập mục A và B.

Theo đề bài, conf(A → B) = Pr(B|A). Trong đó, Pr(B|A) là xác suất có điều kiện của việc tìm thấy B nếu có A, công thức Pr(B|A) = $\frac{Pr(A ∩ B)}{Pr(A)}$. Ví dụ ta có luật kết hợp A -> B, một tiệm tạp hóa bán 20 mặt hàng A và 100 mặt hàng B trong tổng số 1000 mặt hàng. Nhìn vào tử số thấy được Pr(A ∩ B) = $\frac{20}{1000}$ và mẫu số Pr(A) = $\frac{20}{1000}$, nên suy ra Confidence của luật kết hợp này bằng 1 nhưng nó lại không thể hiện mối quan hệ có ý nghĩa giữa mặt hàng A và mặt hàng B, mặt khác nó chỉ cho thấy B xuất hiện nhiều hơn A, mà Pr(B) =$\frac{100}{1000}$ đã bị bỏ qua trong công thức.

Trong khi Lift và Conviction lại có xác suất của cả 2 tập mục A và B trong công thức lift(A → B) = $\frac{conf(A → B)}{S(B)}$ (conf có Pr(A) và S(B) = Pr(B)), tương tự với công thức conv, conv(A → B) = $\frac{(1 − S(B)}{1 − conf(A → B)}$. Lấy một ví dụ minh họa về Lift và số liệu từ phần trên thì ta được Lift = 10, tức tỉ lệ 10:1, chứng tỏ A và B có mối quan hệ liên quan với nhau. Điều này giúp cho công thức có thể tránh được vấn đề thiên vị so với Confidence (Confidence đã loại bỏ Pr(B) khỏi công thức).

**Câu b:** 

Lift là đối xứng (luật A→B = luật B→A), còn Confidence và Conviction thì không phải. Ta suy ra từ công thức:

lift(A→B) = lift(B→A) = $\frac{Pr(A ∩ B)}{Pr(A) x Pr(B)}$.

1. conf(A→B) = Pr(B|A) và conf(B→A) = Pr(A|B).
2. Pr(A|B) và Pr(B|A) có thể khác nhau.
3. conv dựa trên conf và có tính định hướng.

*Ví dụ:* Nếu ta có các rổ AB, AC, AD, AE thì S(A) = 4/4, S(B) = 1/4 và Pr(A ∩ B) = 1/4. Khi đó conf(A→B) = Pr(B|A) ≠ Pr(A|B) = conf(B→A) vì: ${1/4 \choose 4/4}$ ≠ $\frac{1/44}{4}$. Tương tự, conv(A→B) = $\frac{1 − S(B)}{1 − conf(A→B)}$ ≠ $\frac{1 − S(A)}{1 − conf(B→A)}$ = conv(B→A) vì: $\frac{1 − \frac{1}{4}}{1 − \frac{1}{4}}$ = 1 # $\frac{1 − \frac{4}{4}}{1 − 1}$= inf. Vậy nên, chỉ có công thức Lift thỏa mãn được luật A→B = luật B→A. 

**Câu c:**

Conviction và oConfidence thỏa mãn được trong khi Lift thì không. Nếu B xảy ra mỗi thời điểm A xảy ra (nghĩa là Pr(B|A) = 1) thì:

1. conf(A→B) = 1
2. conv(A→B)→infinity
3. lift(A→B) phụ thuộc vào giá trị của Pr(B) và có thể khác vì B có thể xảy ra trong các giỏ không có A. Ví dụ: Nếu chúng ta có các rổ AB, AB, CD, EF thì Pr(B|A) = 1, S(B) = $\frac{1}{2}$, Pr(D|C) = 1, và S(D) = $\frac{1}{4}$. Sau đó, lift(A→B) = $\frac{1}{\frac{1}{2}}$ = 2 và lift(C→D) = $\frac{1}{\frac{1}{4}}$ = 4. Mặc dù cả hai quy tắc đều là quy tắc 100% nhưng chúng có tỉ lệ Lift khác nhau. 

In [ ]:
from pyspark.sql import SparkSession
from functools import partial
from itertools import combinations

In [ ]:
# Câu d: 
if __name__=='__main__':
    spark = SparkSession.builder.getOrCreate()
    sc = spark.sparkContext
    lines=sc.textFile('/content/browsing.txt')

    frequent_items=lines.flatMap(lambda l:l.split()).map(lambda ele:(ele,1)).\
        reduceByKey(lambda e1,e2:e1+e2).filter(lambda x:x[1]>=100)

    frequent_itemset=frequent_items.collectAsMap()

    frequent_pairs = lines.map(lambda l: l.split()).flatMap(partial(combinations, r=2)).map(lambda pair: sorted(pair)) \
        .map(lambda pair: (tuple(pair), 1)).filter(
        lambda ele: ele[0][0] in frequent_itemset and ele[0][1] in frequent_itemset).reduceByKey(
        lambda p1, p2: p1 + p2).filter(lambda x: x[1] >= 100)

    freq_pairs_count=frequent_pairs.collectAsMap()

    frequent_pairs=frequent_pairs.flatMap(lambda ele:[((ele[0][0],ele[0][1]),ele[1]),((ele[0][1],ele[0][0]),ele[1])])

    frequent_pair_conf=frequent_pairs.map(lambda ele:(ele[0],float(ele[1]/frequent_itemset[ele[0][0]]))).sortBy(lambda x:-x[1])

    frequent_pair_conf.toDF().show(10) 

+--------------------+------------------+
|                  _1|                _2|
+--------------------+------------------+
|{DAI93865, FRO40251}|               1.0|
|{GRO85051, FRO40251}| 0.999176276771005|
|{GRO38636, FRO40251}|0.9906542056074766|
|{ELE12951, FRO40251}|0.9905660377358491|
|{DAI88079, FRO40251}|0.9867256637168141|
|{FRO92469, FRO40251}| 0.983510011778563|
|{DAI43868, SNA82528}| 0.972972972972973|
|{DAI23334, DAI62779}|0.9545454545454546|
|{ELE92920, DAI62779}|0.7326649958228906|
|{DAI53152, FRO40251}| 0.717948717948718|
+--------------------+------------------+
only showing top 10 rows



In [ ]:
# Câu e: 
triples=lines.map(lambda l:l.split()).flatMap(partial(combinations,r=3)).map(lambda triple:tuple(list(sorted(triple))))

triples=triples.map(lambda triple:(triple,1)).reduceByKey(lambda t1,t2:t1+t2).filter(lambda x:x[1]>=100)

freq_triples_conf=triples.flatMap(lambda ele:[(((ele[0][0],ele[0][1]),ele[0][2]),ele[1]),
                                                  (((ele[0][0],ele[0][2]),ele[0][1]),ele[1]),
                                                  (((ele[0][1],ele[0][2]),ele[0][0]),ele[1])])\
.reduceByKey(lambda t1,t2:t1+t2).map(lambda ele:(ele[0],ele[1]/freq_pairs_count[ele[0][0]])).sortBy(lambda x:(-x[1],x[0][0],x[0][1]))\
                        .map(lambda ele:(ele[0][0],ele[0][1],ele[1]))
freq_triples_conf.toDF().show(10) 
 

+--------------------+--------+---+
|                  _1|      _2| _3|
+--------------------+--------+---+
|{DAI23334, ELE92920}|DAI62779|1.0|
|{DAI31081, GRO85051}|FRO40251|1.0|
|{DAI55911, GRO85051}|FRO40251|1.0|
|{DAI62779, DAI88079}|FRO40251|1.0|
|{DAI75645, GRO85051}|FRO40251|1.0|
|{ELE17451, GRO85051}|FRO40251|1.0|
|{ELE20847, FRO92469}|FRO40251|1.0|
|{ELE20847, GRO85051}|FRO40251|1.0|
|{ELE26917, GRO85051}|FRO40251|1.0|
|{FRO53271, GRO85051}|FRO40251|1.0|
+--------------------+--------+---+
only showing top 10 rows



# **Câu 3**

**Câu a, b:**

Giả sử chúng ta muốn xác suất “don't know” tối đa là  e−10 . Giả sử n và m đều rất lớn (nhưng n lớn hơn nhiều so với m hoặc k), hãy đưa ra một xấp xỉ đơn giản cho giá trị nhỏ nhất của k sẽ đảm bảo xác suất này tối đa là  e−10 . Biểu hiện của bạn nên là một hàm của n và m.

Số cột có m phần tử 1 trên tổng số n cột là ${n \choose m}$. Số lượng cột đó mà không có số 1 trong & hàng được chọn là ${n-k \choose m}$.
Xác suất để chọn được cột đó là: $\displaystyle\frac{n-k \choose m}{n \choose m}$.
Do đó, xác suất để đạt được "don't know" khi minhashing cho cột đó là 1 trên tổng số ${n \choose m}$ trường hợp cột, tức là:

P(don't know)=${\displaystyle\frac{(n-k) \choose m}{n\choose m}}$ + $\displaystyle\frac{(n-k)!m!(n-m)!}{m!(n-k-m)!n!}$+ $\displaystyle\frac{(n-k)*(n-k-1)...(n-k-m+1)} {n*(n-1)...(n-m+1)}$
Để giới hạn xác suất này, ta có thể sử dụng bất đẳng thức AM- GM:
$\displaystyle\frac{(n - k) + (n - k - 1) + \cdots + (n - k - m + 1)}{m} \ge \sqrt[m]{(n - k) * (n - k - 1) \cdots (n - k - m + 1}$

Hay:
$(n - k - m + 1)*(n - k - m + 2)\cdots(n - k) \le (\displaystyle\frac{n - k}{m})^m $n^m$

Do đó:

$P(\text{don't know}) = \displaystyle\frac{(n -k)*(n -k-1)\cdots(n -k-m + 1)}{n*(n-1)\cdots(n-m+1)} \le (\displaystyle\frac{n - k}{n})^m$

Vì vậy, xác suất tối đa để đạt được "don't know" là $\left(\frac{n-k}{n}\right)^m$.

Với giá trị $e^{-10}$, ta có: $(\displaystyle\frac{n-k}{n})^m \le e^{-10}$

Tương đương với: $(1 - \displaystyle\frac{k}{n})^m \le e^{-10}$

Do $1-x \le e^{-x}$, ta có: $x^{\displaystyle\frac{-km}{n}} = e^{-10}$

**Câu c:**

Hai cột (tập hợp) là:

$[0  1  0]^{T}$ và $[0  1  1]^{T}$
(b) Giá trị hệ số tương đồng Jaccard giữa S1 và S2 là:

J(S1, S2) = |S1 ∩ S2| / |S1 ∪ S2| = $\frac{1}{2}$
(c) Xác suất để một hoán vị tuần hoàn ngẫu nhiên cho giá trị minhash giống nhau cho cả S1 và S2 là:

P(minhash(S1) = minhash(S2) | cyclic permutations) = $\frac{2}{3}$

Giải thích:

Khi chỉ xem xét các hoán vị tuần hoàn, chỉ có ba hoán vị có thể xảy ra: (1,2,3), (2,3,1), và (3,1,2). Nếu chúng ta bắt đầu chu kỳ tại hàng đầu tiên hoặc thứ hai (1 hoặc 2), giá trị minhash của S1 và S2 sẽ giống nhau. Tuy nhiên, nếu chúng ta bắt đầu chu kỳ tại hàng cuối cùng (3), giá trị minhash sẽ khác nhau. Do đó, xác suất của giá trị minhash giống nhau là $\frac{2}{3}$.

# **Câu 4**

**Câu a:** 

Đối với mỗi 1≤ j ≤ L, chúng ta có:

Pr[x∈T∩Wj] ≤ p${k \choose 2}$ = $\displaystyle\frac{1}{n}$,

Điều đó có nghĩa là xác suất một điểm dữ liệu bất kỳ x nằm cùng trong T và Wj không quá $\displaystyle\frac{1}{n}$. Do đó, chúng ta có thể viết:

E[|T ∩ Wj|] = ∑x∈A, Pr[x∈T ∩ Wj] ≤ ∑x∈A, Pr[x∈T] Pr[x∈Wj] ≤ p${k \choose 2}$ = $\displaystyle\frac{1}{n}$.

Sử dụng tính tuyến tính của kỳ vọng, chúng ta có:
 
E[∑j=1LE[PLj=1 |T ∩ Wj|]] ≤ LE[|T ∩ Wj|] ≤ $\displaystyle\frac{L}{n}$.

Bây giờ, chúng ta có thể áp dụng bất đẳng thức Markov để có được:

Pr[∑j=1L|T ∩ Wj| > 3L] ≤ E[∑j=1L|T ∩ Wj|]/3L ≤ $\displaystyle\frac{1}{3}$.



**Câu b:**

Ta có:

Pr[gj($x^{*}$) = gj(z)] ≥ pk1

Ta có thể sử dụng bất đẳng thức trên để thu được một giới hạn trên cho xác suất rằng gj($x^{*}$) ≠ gj(z), như sau:

Pr[gj($x^{*}$) ≠ gj(z)] ≤ 1 - Pr[gj($x^{*}$) = gj(z)] 

≤ 1 - pk1

= 1 - $p^{\displaystyle\frac{log1}{p2(n)}}$) _ 1 (thay thế pk1 bằng định nghĩa của nó) 

= 1 - $(\displaystyle\frac{1}{n})^{\displaystyle\frac{log1}{p2(n)}}$

= 1 - $(\displaystyle\frac{1}{n})^{p}$
 (với ρ = log($\displaystyle\frac{1}{p1}$) / log($\displaystyle\frac{1}{p2}$))

= 1 - $\displaystyle\frac{1}{L}$

= $\displaystyle\frac{(L-1)}{L}$

Trong đó: k = $\displaystyle\frac{log1}{p2(n)}$, do đó L = $n^{p}$.

Tiếp theo, ta có thể sử dụng bất đẳng thức trên để thu được một giới hạn trên cho xác suất rằng tất cả các gj đều thỏa mãn gj($x^{*}$) ≠ gj(z), như sau:

Pr [∀ 1 ≤ j ≤ L, gj($x^{*}$) ≠ gj(z)] 

≤ ∏_{j=1}^L Pr[gj($x^{*}$) ≠ gj(z)] (sử dụng kết hợp liên minh) 

≤ $(\displaystyle\frac{L-1}{L})^{L}$ 

≤ $e^{-1}$ = $\displaystyle\frac{1}{e}$ 

**Câu c:**

Giả sử $x^{*}$ là hàng xóm gần nhất thực sự với điểm truy vấn z với khoảng cách d ($x^{*}$, z) ≤ λ. Từ phần (b), chúng ta biết rằng với xác suất ít nhất là 1 - $\displaystyle\frac{1}{e}$, $x^{*}$ được băm vào ít nhất một trong L thùng mà điểm truy vấn băm vào, tức là tồn tại một hàm băm gj sao cho gj ($x^{*}$) = gj (z).
Bây giờ, hãy để Wj là tập hợp các điểm dữ liệu được ánh xạ đến cùng một thùng như điểm truy vấn z bằng cách sử dụng hàm băm gj, như được định nghĩa trong phần (a). Hãy để T là tập hợp các điểm dữ liệu cách điểm truy vấn xa hơn cλ. Từ phần (a), chúng ta biết rằng xác suất của ít nhất 3L điểm dữ liệu nằm trong phần giao của T và Wj cho một số j nhỏ hơn hoặc bằng $\displaystyle\frac{1}{3}$.

Giả sử rằng $x^{*}$ được băm vào ít nhất một trong L thùng mà điểm truy vấn băm vào, chúng ta cần xem xét hai trường hợp:

Trường hợp 1: $x^{*}$ không nằm trong phần giao của T và bất kỳ Wj nào. Trong trường hợp này, thuật toán sẽ trả về $x^{*}$, đó là hàng xóm gần nhất thực sự với điểm truy vấn với khoảng cách d ($x^{*}$, z) ≤ λ.

Trường hợp 2: $x^{*}$ nằm trong phần giao của T và một số Wj. Trong trường hợp này, thuật toán có thể trả về một điểm không phải là hàng xóm gần nhất thực sự với điểm truy vấn. Tuy nhiên, xác suất của việc này có thể được giới hạn như sau:
Giả sử Yj là biến chỉ mục có giá trị bằng 1 nếu có ít nhất 3 điểm dữ liệu nằm trong giao của T và Wj, và bằng 0 nếu không. Khi đó, xác suất thất bại của thuật toán có thể được biểu diễn như sau:
Pr[thuật toán thất bại] = Pr[$x^{*}$ nằm trong giao của T và một số Wj] × Pr[thuật toán chọn một điểm không phải $x^{*}$ từ giao này]
Sử dụng bất đẳng thức liên hợp và kết quả từ phần (a), ta có:
Pr[$x^{*}$ nằm trong giao của T và một số Wj] ≤ Σj Pr[Yj = 1] ≤ Σj Pr[3L điểm dữ liệu nằm trong T và Wj] ≤ L($\displaystyle\frac{1}{3}$) = ρlog($\displaystyle\frac{1}{p1}$)
Ở đây, bất đẳng thức cuối cùng được suy ra từ định nghĩa của ρ trong thủ tục.
Giờ đây, giả sử Zj là biến chỉ mục có giá trị bằng 1 nếu thuật toán chọn một điểm không phải $x^{*}$ từ giao của T và Wj, và bằng 0 nếu không. Khi đó, xác suất chọn một điểm không phải $x^{*}$ từ giao của T và Wj có thể được biểu diễn như sau:
Pr[thuật toán chọn một điểm không phải $x^{*}$ từ giao này] = Σj Pr[Zj = 1]
Vì các điểm dữ liệu được chọn đồng đều ngẫu nhiên từ giao của T và Wj, nên xác suất chọn một điểm không phải $x^{*}$ bị chặn dưới bởi (|T ∩ Wj| - 1) / |T ∩ Wj|. Do đó, ta có:
Pr[Zj = 1] ≤ (|T ∩ Wj| - 1) / |T ∩ Wj| ≤ 1 - 1/|T ∩ Wj|
Sử dụng bất đẳng thức liên hợp một lần nữa, ta có:
Pr[thuật toán thất bại] ≤ Pr[$x^{*}$ nằm trong giao của T và một số Wj] × Pr[thuật toán chọn một điểm không phải $x^{*}$ từ giao này]

In [ ]:
import numpy as np
#import random
import time
#import pdb
import unittest
from PIL import Image

def l1(u, v):
    return  np.abs(u-v).sum()

In [ ]:
def load_data(filename):
    return np.genfromtxt(filename, delimiter=',')

def create_function(dimensions, thresholds):
    def f(v):
        boolarray = [v[dimensions[i]] >= thresholds[i] for i in range(len(dimensions))]
        return "".join(map(str, map(int, boolarray)))
    return f

In [ ]:
def create_functions(k, L, num_dimensions=400, min_threshold=0, max_threshold=255):
    functions = []
    for i in range(L):
        dimensions = np.random.randint(low = 0, 
                                   high = num_dimensions,
                                   size = k)
        thresholds = np.random.randint(low = min_threshold, 
                                   high = max_threshold + 1, 
                                   size = k)

        functions.append(create_function(dimensions, thresholds))
    return functions

In [ ]:
def hash_vector(functions, v):
    return np.array([f(v) for f in functions])

In [ ]:
def hash_data(functions, A):
    return np.array(list(map(lambda v: hash_vector(functions, v), A)))


In [ ]:
def get_candidates(hashed_A, hashed_point, query_index):
    return filter(lambda i: i != query_index and \
        any(hashed_point == hashed_A[i]), range(len(hashed_A)))


In [ ]:
def lsh_setup(A, k = 24, L = 10):
    functions = create_functions(k = k, L = L)
    hashed_A = hash_data(functions, A)
    return (functions, hashed_A)

In [ ]:
def lsh_search(A, hashed_A, functions, query_index, num_neighbors = 10):
    hashed_point = hash_vector(functions, A[query_index, :])
    candidate_row_nums = get_candidates(hashed_A, hashed_point, query_index)
    
    distances = map(lambda r: (r, l1(A[r], A[query_index])), candidate_row_nums)
    best_neighbors = sorted(distances, key=lambda t: t[1])[:num_neighbors]

    return [t[0] for t in best_neighbors]

In [ ]:
def plot(A, row_nums, base_filename):
    for row_num in row_nums:
        patch = np.reshape(A[row_num, :], [20, 20])
        im = Image.fromarray(patch)
        if im.mode != 'RGB':
            im = im.convert('RGB')
        im.save(base_filename + "-" + str(row_num) + ".png")

In [ ]:
def linear_search(A, query_index, num_neighbors):
    distances=  ( (r, l1(A[r], A[query_index])) for r in   range(len(A))  if r != query_index )
    best_neighbors = sorted(distances, key=lambda t: t[1])[:num_neighbors]
    
    return [t[0] for t in best_neighbors]

In [ ]:
from functools import reduce

def measure_error(A,query_index,lsh_result: np.ndarray,linear_result:np.ndarray):
    def sub_measure(result): 
        return reduce(lambda a,b: a+b,
                      map(lambda r:  l1(A[r],A[query_index]),result),0. )
   
    return sub_measure(lsh_result)/sub_measure(linear_result)

In [ ]:
from matplotlib import pyplot as plt

def plot_error_change(file_path):   
    A= load_data(file_path)
    def geterror(k=24,L =10):
        error =0.
        functions, hashed_A = lsh_setup(A, k = k, L = L)
        for query_index in range(100,1100,100):
            lsh_result =  lsh_search(A, hashed_A, functions, query_index, num_neighbors = 3)
            linear_result =  linear_search( A,query_index,num_neighbors=3)
            error += measure_error(A,query_index,lsh_result,linear_result)
        return error/10
        plot(A,lsh_result,'LSH')
        plot(A,linear_result,'LINEAR')  
        plot(A,[100],'RAW')
    
  
    x_L= range(10,22,2)
    x_K= range(16,26,2)
    y_L= [ geterror(L=l)  for l in x_L ]
    y_K= [geterror(k=k)  for k in x_K  ]
    plt.subplot(2,1,1)
    plt.title('K=24')
    plt.xlabel("L")
    plt.ylabel("error")
    plt.plot(x_L,y_L)
    
    plt.subplot(2,1,2)
    plt.title('L=10')
    plt.xlabel("K")
    plt.ylabel("error")
    plt.plot(x_K,y_K)
    plt.show()
    return (y_L,y_K)
    


In [ ]:
def problem4():
    y_L ,y_K  =  plot_error_change ('/content/sample_data/patches.csv') 
    return  y_L ,y_K 

In [ ]:
class TestLSH(unittest.TestCase):
    def test_l1(self):
        u = np.array([1, 2, 3, 4])
        v = np.array([2, 3, 2, 3])
        self.assertEqual(l1(u, v), 4)

    def test_hash_data(self):
        f1 = lambda v: sum(v)
        f2 = lambda v: sum([x * x for x in v])
        A = np.array([[1, 2, 3], [4, 5, 6]])
        self.assertEqual(f1(A[0,:]), 6)
        self.assertEqual(f2(A[0,:]), 14)

        functions = [f1, f2]
        self.assertTrue(np.array_equal(hash_vector(functions, A[0, :]), np.array([6, 14])))
        self.assertTrue(np.array_equal(hash_data(functions, A), np.array([[6, 14], [15, 77]])))
        if __name__ == '__main__':
          problem4() 
           
